<a href="https://colab.research.google.com/github/ryo-nakajima/the_effect_test/blob/master/uth_bert_medpeer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 必須: ライブラリのインストール

In [ ]:
!pip install transformers  # Hugging Face Transformersライブラリ
!pip install mecab-python3  # MeCabのPythonバインディング


UTH-BERTモデルのダウンロードとセットアップ

In [ ]:
!wget https://ai-health.m.u-tokyo.ac.jp/labweb/dl/uth_bert/UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
!unzip UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip
!rm UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K_pytorch.zip

MeCabとNEologdのセットアップ

In [ ]:
!apt install mecab libmecab-dev mecab-ipadic-utf8 file
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -a -y

 MeCabの初期化

In [ ]:
import MeCab
import subprocess

# NEologdのパスを取得
cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
neologd_dic_dir_path = subprocess.check_output(cmd, shell=True).decode('utf-8').strip()

# MeCabを初期化
mecab = MeCab.Tagger(f"-d {neologd_dic_dir_path}")


万病辞書

In [ ]:
!wget http://sociocom.jp/~data/2018-manbyo/data/MANBYO_201907_Dic-utf8.dic


トークナイザ関数とモデルロード

In [ ]:
from transformers import BertTokenizer, BertModel

# モデルのパスを指定
model_path = "./UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K"

# トークナイザとモデルをロード
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)


実行するコード部分

In [ ]:
def tokenize_with_mecab(text):
    text = mecab.parse(text)
    return text.splitlines()

def get_cls_embedding(text, model, tokenizer):
    tokens = tokenize_with_mecab(text)
    encoded = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")
    model.eval()
    with torch.no_grad():
        outputs = model(**encoded)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze(0)

def calculate_cosine_similarity(embedding1, embedding2):
    similarity = F.cosine_similarity(embedding1, embedding2, dim=0)
    return similarity.item()


類似性の計算

In [ ]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

# モデルのパス
model_path = "./UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K"

# トークナイザとモデルをロード
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

# CSVファイルを読み込む
file_path = "./data/comments_physicians.csv"
df = pd.read_csv(file_path)

# コメントの特徴ベクトルを取得する関数
def get_cls_embedding(text, model, tokenizer):
    # MeCabで形態素解析
    import MeCab
    mecab = MeCab.Tagger("-Owakati")
    tokens = mecab.parse(text).strip().split()

    # トークナイザでトークナイズ
    encoded = tokenizer(tokens, is_split_into_words=True, return_tensors="pt")

    # モデルを推論モードに設定して推論
    model.eval()
    with torch.no_grad():
        outputs = model(**encoded)

    # [CLS]トークンの埋め込みを取得
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLSトークン: 位置0
    return cls_embedding.squeeze(0)  # ベクトルの形状を整える

# コサイン類似度を計算する関数
def calculate_cosine_similarity(embedding1, embedding2):
    similarity = F.cosine_similarity(embedding1, embedding2, dim=0)
    return similarity.item()

# 任意の行を指定して類似度を計算
row1, row2 = 0, 1  # 計算したい2つの行のインデックスを指定
comment1 = df.loc[row1, "comment"]
comment2 = df.loc[row2, "comment"]

# コメントの埋め込みを取得
embedding1 = get_cls_embedding(comment1, model, tokenizer)
embedding2 = get_cls_embedding(comment2, model, tokenizer)

# 類似度を計算
similarity = calculate_cosine_similarity(embedding1, embedding2)

# 結果を表示
print(f"Similarity between row {row1} and row {row2}: {similarity}")
